In [186]:
import pandas as pd
df_matches = pd.read_csv("matches.csv")
df_matches.head(3)

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN


In [221]:
df_deliveries = pd.read_csv("deliveries.csv")
df_deliveries.head(3)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN


In [190]:
print(df_matches.shape)
print(df_deliveries.shape)

(756, 18)
(179078, 21)


In [191]:
#plotting every team wins not only RCB and KKR. 
import matplotlib.pyplot as plt
import plotly.graph_objs as go
df_winner_matches = df_matches[['id','winner']].groupby('winner').size().reset_index()
df_winner_matches.columns = ['winner','total_wins']
df_winner_matches = df_winner_matches.sort_values('total_wins',ascending=False)

data = go.Bar(x = df_winner_matches['winner'],y = df_winner_matches['total_wins'])
layout = go.Layout(title = 'Total wins by every team in IPL',xaxis = dict(title = 'Team'),yaxis = dict(title = 'No of wins'))
fig = go.Figure(data = data,layout = layout)
fig.show()

In [192]:
print(df_matches.columns)
print(df_deliveries.columns)

Index(['id', 'season', 'city', 'date', 'team1', 'team2', 'toss_winner',
       'toss_decision', 'result', 'dl_applied', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue', 'umpire1', 'umpire2',
       'umpire3'],
      dtype='object')
Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')


In matches data frame columns like umpire1,umpire2,umpire3 are not useful because these days they are using technology. So we cant decide which umpire is more useful for predicting anything related to match.

Date is also not useful because we already have season column which is also almost equal to date.

City and venue also not important features for this year IPL. Because IPL2020 is happening in UAE, only 2014 ipl was happened in UAE that to half season.

In [193]:
df_matches = df_matches.drop(['city','venue','date','umpire1','umpire2','umpire3'], axis = 1) 
df_matches.head(2)

,id,season,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match
0,1,2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh
1,2,2017,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith


In [194]:
print(df_matches["team2"].value_counts())

Royal Challengers Bangalore    95
Kolkata Knight Riders          95
Delhi Daredevils               89
Mumbai Indians                 86
Kings XI Punjab                85
Rajasthan Royals               80
Chennai Super Kings            75
Sunrisers Hyderabad            45
Deccan Chargers                32
Pune Warriors                  26
Gujarat Lions                  16
Delhi Capitals                 10
Rising Pune Supergiant          8
Rising Pune Supergiants         7
Kochi Tuskers Kerala            7
Name: team2, dtype: int64


In [222]:
#In 2019 delhi daredevils name changed to delhi capitals. So i am replacing old name with new name for all the rows.
df_matches = df_matches.replace(to_replace ="Delhi Daredevils", value = "Delhi Capitals") 

df_deliveries = df_deliveries.replace(to_replace = "Delhi Daredevils", value = "Delhi Capitals")

In [196]:
# We are predicting results of the match between SRH and CSK. So i am taking only rows which has SRH or CSK from team1 and team2 columns in matches dataset.
df_matches = df_matches[(df_matches['team1'] == 'Sunrisers Hyderabad') | (df_matches['team1'] == 'Chennai Super Kings') | 
                        (df_matches['team2'] == 'Sunrisers Hyderabad') | (df_matches['team2'] == 'Chennai Super Kings')]


df_matches.head(2)

,id,season,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match
0,1,2017,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh
5,6,2017,Gujarat Lions,Sunrisers Hyderabad,Sunrisers Hyderabad,field,normal,0,Sunrisers Hyderabad,0,9,Rashid Khan


In [197]:
#checking null values in matches dataset
print(df_matches.isnull().sum())

id                 0
season             0
team1              0
team2              0
toss_winner        0
toss_decision      0
result             0
dl_applied         0
winner             0
win_by_runs        0
win_by_wickets     0
player_of_match    0
dtype: int64


No null values present in matches dataset for srh and csk matches.

In [223]:
#checking null values in deliveries dataset
print(df_deliveries.isnull().sum())

match_id                 0
inning                   0
batting_team             0
bowling_team             0
over                     0
ball                     0
batsman                  0
non_striker              0
bowler                   0
is_super_over            0
wide_runs                0
bye_runs                 0
legbye_runs              0
noball_runs              0
penalty_runs             0
batsman_runs             0
extra_runs               0
total_runs               0
player_dismissed    170244
dismissal_kind      170244
fielder             172630
dtype: int64


Player_dismissed, dismissal_kind and fielder are having null values. But if you consider today questions only player_dismissed is useful for calculating average of batsman. So i am removing remaining two features.

Filling player_dismissed null values with no wicket

In [224]:
#dropping columns
df_deliveries = df_deliveries.drop(['dismissal_kind','fielder'], axis = 1) 
#filling null values
df_deliveries["player_dismissed"].fillna("No Wicket", inplace = True)
df_deliveries.head(2)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,No Wicket
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,No Wicket


## Exploratory Data analysis(EDA)

### Q1

In [200]:
#plotting SRH and CSK wins  against all teams
import plotly.graph_objs as go
df_winner = df_matches[['id','winner']].groupby('winner').size().reset_index()
df_winner.columns = ['winner','total_wins']
df_winner = df_winner.sort_values('total_wins',ascending=False)[:2]

data = go.Bar(x = df_winner['winner'],y = df_winner['total_wins'])
layout = go.Layout(title = 'Total wins by SRH and CSK in IPL',xaxis = dict(title = 'Team'),yaxis = dict(title = 'No of wins'))
fig = go.Figure(data = data,layout = layout)
fig.show()

In [201]:
df_total_wins = df_matches[['id','winner']].groupby('winner').size().reset_index().sort_values(0,ascending = False)[:2]
df_total_wins.columns = ['Team','Total_wins']

df_total_wins

,Team,Total_wins
0,Chennai Super Kings,100
12,Sunrisers Hyderabad,58


Actually SRH is playing IPL from 2013.So they missed 5 seasons. But CSK also not played 2seasons(2016-17). Overall CSK played 3 more seasons than SRH. So above obsrvation is not that much useful.

In [202]:
# Lets check winning ratio between SRH VS CSK
df_CSK_SRH = df_matches[((df_matches['team1'] == 'Sunrisers Hyderabad') | (df_matches['team1'] == 'Chennai Super Kings')) & 
                        ((df_matches['team2'] == 'Sunrisers Hyderabad') | (df_matches['team2'] == 'Chennai Super Kings'))] 

#plotting SRH and CSK wins against each other
import plotly.graph_objs as go
df_winner = df_CSK_SRH[['id','winner']].groupby('winner').size().reset_index()
df_winner.columns = ['winner','total_wins']
df_winner = df_winner.sort_values('total_wins',ascending=False)[:2]

data = go.Bar(x = df_winner['winner'],y = df_winner['total_wins'])
layout = go.Layout(title = 'Total wins by SRH and CSK in IPL',xaxis = dict(title = 'Team'),yaxis = dict(title = 'No of wins'))
fig = go.Figure(data = data,layout = layout)
fig.show()                         

CSK won more matches than SRH in IPL till now but this is not enough to predict todays match results. Because players will change every 2 years. So its better look individual player runs or wickets against each other.

In [203]:
df_SRHbatsman_CSKbowlers = df_deliveries[((df_deliveries['batsman'] == 'DA Warner') | (df_deliveries['batsman'] == 'J Bairstow') |
                             (df_deliveries['batsman'] == 'MK Pandey') | (df_deliveries['bowler'] == 'DL Chahar') | 
                             (df_deliveries['bowler'] == 'SN Thakur') | (df_deliveries['bowler'] == 'RA Jadeja'))] 

print("SRH top 3 batsman runs against CSK bowlers : " , df_SRHbatsman_CSKbowlers['batsman_runs'].sum())
SRH_bat_average = df_SRHbatsman_CSKbowlers['batsman_runs'].sum()/len(df_SRHbatsman_CSKbowlers[(df_SRHbatsman_CSKbowlers['player_dismissed'] == 'DA Warner') |
                                                                                              (df_SRHbatsman_CSKbowlers['player_dismissed'] == 'J Bairstow')|
                                                                                              (df_SRHbatsman_CSKbowlers['player_dismissed'] == 'MK Pandey')])
print("SRH batsman batting average against CSK bowlers : ",SRH_bat_average)

df_CSKbatsman_SRHbowlers= df_deliveries[((df_deliveries['batsman'] == 'SR Watson') | (df_deliveries['batsman'] == 'F du Plessis') |
                             (df_deliveries['batsman'] == 'MS Dhoni') | (df_deliveries['bowler'] == 'Rashid Khan') |
                             (df_deliveries['bowler'] == 'K Ahmed') | (df_deliveries['bowler'] == 'S Sharma'))]

print("CSK top 3 batsman runs against SRH bowlers : " , df_CSKbatsman_SRHbowlers['batsman_runs'].sum())
CSK_bat_average = df_CSKbatsman_SRHbowlers['batsman_runs'].sum()/len(df_CSKbatsman_SRHbowlers[(df_CSKbatsman_SRHbowlers['player_dismissed'] == 'SR Watson') |
                                                                                              (df_CSKbatsman_SRHbowlers['player_dismissed'] == 'F du Plessis')|
                                                                                              (df_CSKbatsman_SRHbowlers['player_dismissed'] == 'MS Dhoni')])
print("CSK batsman batting average against SRH bowlers : ",CSK_bat_average)


SRH top 3 batsman runs against CSK bowlers :  12914
SRH batsman batting average against CSK bowlers :  61.203791469194314
CSK top 3 batsman runs against SRH bowlers :  11699
CSK batsman batting average against SRH bowlers :  43.169741697416974


Batting average of CSK batsman against SRH bowlers are having less average than SRH batsman against CSK bowlers. 

So i am predicting SRH will win this match even though previous record is in CSK favour. But for this question i am going with individual player records.

### Q2

In [204]:
df_SRHCSK_bowlers = df_deliveries[((df_deliveries['bowler'] == 'Rashid Khan') | (df_deliveries['bowler'] == 'K Ahmed') |
                             (df_deliveries['bowler'] == 'S Sharma') | (df_deliveries['bowler'] == 'DL Chahar') | 
                             (df_deliveries['bowler'] == 'SN Thakur') | (df_deliveries['bowler'] == 'RA Jadeja') |
                             ((df_deliveries['bowler'] == 'DJ Bravo')))] 
print("Number of wide balls bowled by present SRH and CSK bowlers : ",df_SRHCSK_bowlers['wide_runs'].sum()) 
print("Number of balls bowled by SRH and CSK bowlers : ", df_SRHCSK_bowlers.shape[0])                           

Number of wide balls bowled by present SRH and CSK bowlers :  292
Number of balls bowled by SRH and CSK bowlers :  8337


In [205]:
number_of_overs = df_SRHCSK_bowlers.shape[0]/6
print("number of overs bowled by SRH and CSK bowlers",number_of_overs)
widee_ball_per_over = int(number_of_overs/df_SRHCSK_bowlers['wide_runs'].sum())
print("For every",+widee_ball_per_over," overs they bowled wide ball")

number of overs bowled by SRH and CSK bowlers 1389.5
For every 4  overs they bowled wide ball


In [207]:
#So we have total 40 overs in the match
wide_balls_this_match = 40/widee_ball_per_over
print("wide balls bowled by CSK and SRH bowlers per match",wide_balls_this_match)

wide balls bowled by CSK and SRH bowlers per match 10.0


I am predicting in today match bowlers will bowl 9 or more wide balls.

### Q3

In [208]:
df_Warner = df_deliveries[(((df_deliveries['batsman'] == 'DA Warner')) & ((df_deliveries['bowler'] == 'SN Thakur') |
                                                                        (df_deliveries['bowler'] == 'DL Chahar') | 
                                                                        (df_deliveries['bowler'] == 'RA Jadeja') |
                                                                        (df_deliveries['bowler'] == 'DJ Bravo')))] 
#Note : i am taking only present CSK bowlers not all CSK bowlers                                                                        
print("David Warner runs against present CSK bowlers : " , df_Warner['batsman_runs'].sum())
print("In Number of matches David Warner got batting aginst present CSK bowlers : ",df_Warner['match_id'].nunique())
print("In Number of matches David Warner out against present CSK bowlers : ",len(df_Warner[df_Warner['player_dismissed'] == 'DA Warner']))                                                                      

David Warner runs against present CSK bowlers :  173
In Number of matches David Warner got batting aginst present CSK bowlers :  13
In Number of matches David Warner out against present CSK bowlers :  2


In [209]:
#By looking at above observations David Warner is remained not out on 11 matches out of 13 played against CSK bowlers
#Now i am claculating Warner batting average against CSK bowlers
Warner_average = df_Warner['batsman_runs'].sum()/len(df_Warner[df_Warner['player_dismissed'] == 'DA Warner'])
print("David warner average against RCB bowlers : ",Warner_average)

David warner average against RCB bowlers :  86.5


David warner batting average is very good against CSK bowlers. So i am predicting he will score 40 or more runs in today match

### Q4

In [231]:
#I am taking SRH bowlers then calcualting number of wickets they took till now per match in IPL
df_SRH_bowlers= df_deliveries[((df_deliveries['bowler'] == 'Rashid Khan') |(df_deliveries['bowler'] == 'K Ahmed'))]
                             
# dfd =df_SRH_bowlers[(df_SRH_bowlers['player_dismissed'] != 'No Wicket')]
print("Number of wickets took by two main SRH bowlers ipl till now: " , (df_SRH_bowlers['player_dismissed'] != 'No Wicket').sum())
print("Number of matches played by both SRH bowlers : ",df_SRH_bowlers['match_id'].nunique())

Number of wickets took by two main SRH bowlers ipl till now:  80
Number of matches played by both SRH bowlers :  46


So for every match both bowlers took atleast 2 wickets. But there are total  bowlers will bowl in this match. So i am predicting option 5 0r 6-8 depending on the batting order(domain knowledge)

### Q5

In [236]:
CSK_SRH_batting = df_deliveries[((df_deliveries['batting_team'] == 'Sunrisers Hyderabad') | (df_deliveries['batting_team'] == 'Chennai Super Kings'))]
                             
print("number of runs scored by CSK and SRH teams till now in ipl",CSK_SRH_batting['batsman_runs'].sum()) 
print("number of matches played by CSK and SRH teams till now in ipl",CSK_SRH_batting['match_id'].nunique()) 

number of runs scored by CSK and SRH teams till now in ipl 41354
number of matches played by CSK and SRH teams till now in ipl 260


In [237]:
CSK_SRH_average = CSK_SRH_batting['batsman_runs'].sum()/CSK_SRH_batting['match_id'].nunique()
print("number of runs scored in a match by SRH and CSK batsman for only one innings",CSK_SRH_average)

number of runs scored in a match by SRH and CSK batsman 159.05384615384617


Here i considered only unique matches i.e only one innings so if you calculate for both innings then 159 +159 = 358.

So it is almost 360. So i am predicting in today match they will score 361 or more.